In [1]:
import json

from openai import OpenAI

from elasticsearch import Elasticsearch

from tqdm.auto import tqdm

In [2]:
client = OpenAI()

In [3]:
!curl http://localhost:9200

curl: (7) Failed to connect to localhost port 9200 after 0 ms: Couldn't connect to server


In [4]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [5]:
es_client = Elasticsearch('http://localhost:9200')

index_settings_homework = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = 'index_homework'

es_client.indices.create(index=index_name, body=index_settings_homework)

for document in tqdm(documents):
    es_client.index(index=index_name, document=document)

/tmp/ipykernel_124208/3830275662.py:20: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es_client.indices.create(index=index_name, body=index_settings_homework)


ConnectionError: Connection error caused by: ConnectionError(Connection error caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x7f720a1d7c50>: Failed to establish a new connection: [Errno 111] Connection refused))

In [ ]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)

    result_docs = []

    for hit in response['hits']['hits']:
        result_docs.append(hit['_score'])
    
    return result_docs

In [ ]:
query = "How do execute a command on a Kubernetes pod?"
elastic_search(query)


/tmp/ipykernel_74183/3690845546.py:17: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es_client.search(index=index_name, body=search_query)


[43.355465, 34.332695, 33.047203, 32.534035, 31.80385]

In [ ]:
def elastic_search_filter(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)

    result_docs = []

    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [ ]:
query = "How do copy a file to a Docker container?"
elastic_search_filter(query)

/tmp/ipykernel_74183/776466855.py:22: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es_client.search(index=index_name, body=search_query)


[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

In [ ]:
def build_prompt(query, search_results):

    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
    Use only the facts from the CONTEXT to answer. 

    QUESTION: {question}

    CONTEXT: {context}
    """.strip()

    context = ""

    for doc in search_results:
        context += f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"    

    prompt = prompt_template.format(question=query, context=context).strip()

    return prompt

In [6]:
query = "How do copy a file to a Docker container?"

search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

search_results = es_client.search(index=index_name, body=search_query)

/tmp/ipykernel_124208/1163521147.py:23: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  search_results = es_client.search(index=index_name, body=search_query)


ConnectionError: Connection error caused by: ConnectionError(Connection error caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x7f7209edbad0>: Failed to establish a new connection: [Errno 111] Connection refused))

In [7]:
search_results['hits']['hits']

NameError: name 'search_results' is not defined

In [8]:
context_template = """
Q: {question}
A: {text}
""".strip()

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [9]:
context_pieces = []

for hit in search_results['hits']['hits']:
    doc = hit['_source']
    context_piece = context_template.format(**doc)
    context_pieces.append(context_piece)

context = '\n\n'.join(context_pieces)

NameError: name 'search_results' is not defined

In [10]:
prompt = prompt_template.format(question=query, context=context)
len(prompt)

NameError: name 'context' is not defined

In [11]:
def build_prompt(query, search_results):

    context_template = """
    Q: {question}
    A: {text}
    """.strip()

    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.

    QUESTION: {question}

    CONTEXT:
    {context}
    """.strip()    

    context_pieces = []

    for hit in search_results['hits']['hits']:
        doc = hit['_source']
        context_piece = context_template.format(**doc)
        context_pieces.append(context_piece)

    context = '\n\n'.join(context_pieces)

    prompt = prompt_template.format(question=query, context=context)

    return prompt

In [12]:
result = build_prompt(query, search_results)
print(result)

NameError: name 'search_results' is not defined